### DATA 643 Project 5: Implementing a Recommender System on Spark
_Nathan, Angus, Pavan_

_The goal of this project is give you practice beginning to work with a distributed recommender system. It is sufficient for this assignment to build out your application on a single node._

Adapt one of your recommendation systems to work with Apache Spark and compare the performance with your previous iteration. Consider the efficiency of the system and the added complexity of using Spark. You may complete the assignment using PySpark (Python), SparkR (R) , sparklyr (R), or Scala.

Please include in your conclusion: For your given recommender system’s data, algorithm(s), and (envisioned) implementation, at what point would you see moving to a distributed platform such as Spark becoming necessary?

We will build a movie recommender using _collaborative filtering_ with _Spark's Alternating Least Saqures implementation_. For the scope of the project, _ALS_ matrix factorization results will be compared with the SVD matrix factorization we built in Project 3. 

#### Dataset

We will be using [MovieLens Latest Datasets](https://grouplens.org/datasets/movielens/). GroupLens Research maintains movie rating data sets collected from the website [MovieLens](http://movielens.org). The datasets were collected over various periods of time, depending on the size of the set. For the scope of the project, we will be using _small dataset_ that contains 100,000 ratings and 1,300 tag applications applied to 9,000 movies by 670 users. The dataset will be split into a training and test sets, to measure the accuracy of the estimations of training set against the test set.

In [30]:
#Import libraries
import numpy as np
import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
import urllib.request #read dataset from GitHib
import math

#Functions
def split_csv_data(str):
    l = str.split(",")
    return [l[0], l[1], l[2]]

def dropFirstRow(index,iterator):
     return iter(list(iterator)[1:]) if index==0 else iterator

#Initialize the environment
conf = SparkConf().setAppName('ALSTest')
sc = SparkContext.getOrCreate(conf=conf)

#Get the movielens data
#Save the file as local copy
f = urllib.request.urlretrieve ("https://raw.githubusercontent.com/akulapa/Data643-Week02/master/Data/ratings.csv", "ratings.csv")
data_file = "./ratings.csv"
ratings_RDD = sc.textFile(data_file)

#Remove timestamp column from the dataset
ratings_RDD = ratings_RDD.map(lambda line: split_csv_data(line))

#Remove the header from the dataset
ratings_RDD = ratings_RDD.mapPartitionsWithIndex(dropFirstRow)

#Print output
ratings_RDD.take(5)

[['1', '31', '2.5'],
 ['1', '1029', '3.0'],
 ['1', '1061', '3.0'],
 ['1', '1129', '2.0'],
 ['1', '1172', '4.0']]

Dataset is converted into _Resilient Distributed Dataset_, RDD. Data is stored as lists in RDD. First value is _userId_, second is _movieId_ and last one is _rating._

In [31]:
#Convert values in numeric values
ratings= ratings_RDD.map(lambda l: Rating(int(l[0]),int(l[1]),float(l[2])))
ratings.take(5)

[Rating(user=1, product=31, rating=2.5),
 Rating(user=1, product=1029, rating=3.0),
 Rating(user=1, product=1061, rating=3.0),
 Rating(user=1, product=1129, rating=2.0),
 Rating(user=1, product=1172, rating=4.0)]

_Ratings_ function in _pyspark.mllib.recommendation_, converts dataset into named lists.

In [32]:
#Generate datasets
#Split dataset into train, validation and test sets
movie_train, movie_val, movie_test= ratings.randomSplit([0.6, 0.2, 0.2])

#Load data into memory
movie_train.cache()
movie_test.cache()
movie_val.cache()

#Sample results
print('Train set')
movie_train.take(5)

Train set


[Rating(user=1, product=1129, rating=2.0),
 Rating(user=1, product=1172, rating=4.0),
 Rating(user=1, product=1263, rating=2.0),
 Rating(user=1, product=1293, rating=2.0),
 Rating(user=1, product=1339, rating=3.5)]

ALS function takes input parameters

- Training dataset
- rank, is the number of latent or hidden factors in the model. Example, when user1 rates a movie as 4 and user2 rates the same movie as 3 there is some feature of the user that makes them rate in a certain way. It could be age, gender or location, etc. If for sure we know the factors that impact the user we could use the number directly. In this case, it would be 3. Since we don't know the features we can start with 1 and increment is until there is no improvement in the results.
- iterations, ALS starts with a default value and improves one matrix at a time. This parameter defines number times to loop through.

Best way to determine optimal values of _rank_ and _iterations_ is by experimentation. We have split dataset into _train_, _validation_ and _test_ sets. We would 

The ALS function also takes in additional parameters. However, we have not experimented with them.

Due to limited resources, we have experimented _rank_ values between 1 and 8, while _iterations_ between 1 and 5. Beyond these values, we received errors and application would crash.

In [33]:
#Remove ratings for validation and test datasets
movie_test_no_rate = movie_test.map(lambda x: (x[0], x[1]))
movie_val_no_rate = movie_val.map(lambda x: (x[0], x[1]))

#Load data into memory
movie_test_no_rate.cache()
movie_val_no_rate.cache()

PythonRDD[3773] at RDD at PythonRDD.scala:48

In [34]:
# Training the model
rank = 1
iterations = 5

min_error = 0
best_rank = -1

for rank in range(1, 8):
    #generate model
    model = ALS.train(movie_train, rank=rank, iterations=iterations)

    #generate predictions
    predictions = model.predictAll(movie_val_no_rate).map(lambda r: ((r[0], r[1]), r[2]))

    #get actual vs predictions
    rates_and_preds = movie_val.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)

    #calculate error
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())

    #display output
    print('For rank %s the RMSE is %s' % (rank, error))

    if error < min_error:
        best_rank = rank

    min_error = error
        
print('The best model was trained with rank %s' % best_rank)

For rank 1 the RMSE is 2.7641191563759526
For rank 2 the RMSE is 1.0587037064621716
For rank 3 the RMSE is 1.0311751605822177
For rank 4 the RMSE is 1.0592531605329332
For rank 5 the RMSE is 1.119789915769175
For rank 6 the RMSE is 1.121753195059547
For rank 7 the RMSE is 1.1584403886549701
The best model was trained with rank 3


Based on the output _rank_ 3 yeilds lower error and suggests it is better model.

In [36]:
#generate the model with best rank and iterations
rank = 3
iterations = 5
model = ALS.train(movie_train, rank=rank, iterations=iterations)

#apply the model for test data.
predictions = model.predictAll(movie_test_no_rate).map(lambda r: ((r[0], r[1]), r[2]))

# joining the prediction with the original test dataset
ratesAndPreds = movie_test.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)

# calculating error
RMSE = math.sqrt(ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
print("Root Mean Squared Error = " + str(RMSE))

Root Mean Squared Error = 1.0444940254408608


#### Conclusions

- Due to system resource constraints, we could not test for best _iterations_ beyond 5.
- MSE is higher for ALS with parameters(_rank_ = 3 and _iterations_ = 5) compared to SVD. ALS RMSE: 1.04, SVD RMSE : 0.254
- Also, _rank_ 3 does not make much sense as many factors could influence user rating a movie. We could not experiment further due to system resource constraints.
- However, we would still suggest ALS when resources are not of concern.
- To test out ALS for more iterations, we ventured into _databricks_ platform.
- _databricks_ platform is easy to setup and was able to take advantage of the libraries that are readily available.

#### References

- https://medium.com/@connectwithghosh/simple-matrix-factorization-example-on-the-movielens-dataset-using-pyspark-9b7e3f567536
- https://www.codementor.io/jadianes/building-a-recommender-with-apache-spark-python-example-app-part1-du1083qbw
- https://dataplatform.cloud.ibm.com/exchange/public/entry/view/5ad1c820f57809ddec9a040e37b2bd55
- https://stackoverflow.com/questions/24718697/pyspark-drop-rows
- https://stackoverflow.com/questions/30729656/what-is-rank-in-als-machine-learning-algorithm-in-apache-spark-mllib